In [1]:
using Pkg;
Pkg.activate(".")
Pkg.add("BenchmarkTools")
Pkg.status()

using BenchmarkTools;

  Activating environment at `~/Documents/GitHub/hello-julia/src/Functors/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/hello-julia/src/Functors/Project.toml`
  No Changes to `~/Documents/GitHub/hello-julia/src/Functors/Manifest.toml`


      Status `~/Documents/GitHub/hello-julia/src/Functors/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0


# Exploring functors

Idea came from the `Polynomial` `struct` in the [MIT website]().

In [2]:
struct Polynomial{R}
    coeffs::Vector{R}
end

function (p::Polynomial)(x)
    v = p.coeffs[end]
    for i = (length(p.coeffs)-1):-1:1
        v = v*x + p.coeffs[i]
    end
    return v
end

(p::Polynomial)() = p(0) #default polynomial value

In [3]:
p = Polynomial([1,2,1])

Polynomial{Int64}([1, 2, 1])

In [4]:
p(4)

25

In [5]:
rp = Polynomial(rand(4))

Polynomial{Float64}([0.05457974939845678, 0.055655068901082405, 0.35287705813802894, 0.8616331244167144])

In [6]:
p() #uses the default Polynomial functor.

1

In [7]:
p.coeffs[1] = 2

2

In [8]:
p.coeffs

3-element Vector{Int64}:
 2
 2
 1

# Functor as simple integrator

We can utilize functor construction to provide us a way to generate a kind of object-oriented version of the integrator.
Let's try the RK4 version [^1].

Later on we can move to a generalized integrator via the Butcher's Tableu.

---
This is under the `<hline>`.

[^1]: We shall implement a simple RK4 version here.

---
``\LaTeX`` equations can be a code within the code.
``E = mc^2``

In [9]:
struct Integrator{T}
    K::Vector{T}
    dt::Vector{T}
end

RK4Integrator = Integrator([0.0,0.0,0.0],[0.0])
EulerStep = Integrator([0.0],[0.0])

Integrator{Float64}([0.0], [0.0])

In [10]:
"""

```
f :: f(u,t,p)
```
"""
function (quad::Integrator)(f::Function, u0, tspan::Tuple{Real,Real}, p::Vector; steps=1)
    quad.dt[1] = (tspan[2] - tspan[1]) / steps;
    result = u0;
    quad.K[1] = quad.dt[1] * f(u0, tspan[1], p[:]);
    return result + quad.K[1];
end

Integrator

In [11]:
euler = Integrator([0.0],[0.1])

Integrator{Float64}([0.0], [0.1])

In [12]:
euler.dt

1-element Vector{Float64}:
 0.1

In [13]:
f(u,t,p) = p[1] * u + p[2] * t

f (generic function with 1 method)

In [14]:
euler(f, 1.0, (0.0,0.1), [-1.0,0.0])

0.9

In [15]:
function euler_step(f::Function, u0, tspan::Tuple{Real,Real}, p::Vector; steps=1)
    dt = (tspan[2] - tspan[1]) / steps;
    result = u0;
    K1 = dt * f(u0, tspan[1], p[:]);
    return result + K1;
end

euler_step (generic function with 1 method)

In [16]:
euler_step(f, 1.0, (0.0,0.1), [-1.0,0.0])

0.9

In [17]:
bmark_es = @benchmark for _ in 1:100_000 euler_step(f, 1.0, (0.0,0.1), [-1.0,0.0]) end

BenchmarkTools.Trial: 667 samples with 1 evaluation.
 Range (min … max):  5.773 ms … 10.469 ms  ┊ GC (min … max):  0.00% … 20.34%
 Time  (median):     7.165 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.491 ms ±  1.052 ms  ┊ GC (mean ± σ):  10.27% ± 10.62%

      ▂▂▂   ▃▂ ▁▂▁█▃▃▃▃          ▁         ▁  ▁   ▁           
  ▄▄▄██████▆██▇█████████▆▄▃▃▃▅▄▄▃██▆▆█▇▆█▅██▇▅█████▆▅▂▄▃▂▃▁▃ ▅
  5.77 ms        Histogram: frequency by time        9.71 ms <

 Memory estimate: 18.31 MiB, allocs estimate: 200000.

In [18]:
bmark_e0 = @benchmark for _ in 1:100_000 euler(f, 1.0, (0.0,0.1), [-1.0,0.0]) end

BenchmarkTools.Trial: 215 samples with 1 evaluation.
 Range (min … max):  18.429 ms … 29.035 ms  ┊ GC (min … max):  0.00% … 13.72%
 Time  (median):     23.355 ms              ┊ GC (median):    14.96%
 Time  (mean ± σ):   23.271 ms ±  2.298 ms  ┊ GC (mean ± σ):  10.40% ±  7.15%

                              ▂▆▃ █ ▃                          
  ▃▁▅▁▆▅▇▅▄▅▅▇▃▄▅█▁▅▁▅▄▄▃▄▇▇▆▅███▇█▅█▄▆▇█▇▅▇▄▇█▄▇▃▇▆▅▃▄▅▄▇▆▄▄ ▄
  18.4 ms         Histogram: frequency by time        27.4 ms <

 Memory estimate: 28.99 MiB, allocs estimate: 700000.

In [19]:
median(bmark_e0.times) / median(bmark_es.times)

3.2594749269173366

In [20]:
bmark_e0.memory / bmark_es.memory

1.5833333333333333

# CONCLUSION

!!! warning "Important notes"

 1. It is still best to use function as much as possible rather than functors.
 2. The function implementation --at least for Euler step-- is much faster and more efficient.
    - faster by at least 3 times
    - memory efficient by at least 1.5 times